In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read Data
uni = pd.read_csv("../data/universities.csv")
prof = pd.read_csv("../data/professors.csv", index_col=0)
article = pd.read_pickle("../data/clean_articles.pkl")

In [8]:
prof

,id,name,university,user_id,affiliation,v_email_at,cited_by,intrests,department
0,0,Ali Fahim,University of Tehran,_C4Iif8AAAAJ,"Assistant Professor of Physics, University of ...",ut.ac.ir,121695,"['Elementary Particle Physics', 'Physics of Co...",Physics
1,1,Mohammad Reza Ganjali,University of Tehran,IJ3XCecAAAAJ,"Professor of Analytical Chemistry, University ...",khayam.ut.ac.ir,49589,"['Sensor and biosensor', 'Elctroanalytical Che...",Chemistry
2,2,Norouzi Parviz,University of Tehran,usWI6hMAAAAJ,"Professor of Electrochemistry, Collage of Scie...",ut.ac.ir,24867,"['Electrochemistry', 'Electrochemical Sensors'...",Biology
3,3,Reza Tavakkoli-Moghaddam,University of Tehran,gqVUx4cAAAAJ,Distinguished Professor of Industrial Engineer...,ut.ac.ir,23651,"['Facilities design', 'Supply chain', 'Schedul...",History
4,4,Ali Akbar Moosavi-Movahedi,University of Tehran,sZysKwIAAAAJ,University of Tehran,ut.ac.ir,16918,['Biophysical Chemistry'],Chemistry
...,...,...,...,...,...,...,...,...,...
49389,49389,Farnaz Hadizadeh,Mazandaran University of Science and Technology,SsISRdQAAAAJ,"Electronic engineer, Mazandaran university of ...",ustmb.ac.ir,6,"['Nanoelectronic', 'Optoelectronic', 'Nanostru...",Physics
49390,49390,masoud askari,Mazandaran University of Science and Technology,ARkaB34AAAAJ,phd candidate at MUST,ustmb.ac.ir,3,['agent based modeling and simulation'],Computer Science
49391,49391,Sina Gouran-Orimi,Mazandaran University of Science and Technology,81ZTghUAAAAJ,Mazandaran University of Science and Technology,ustmb.ac.ir,3,"['Power Systems Planning', 'LFC', 'Distributed...",History
49392,49392,Tarnaz Chamani,Mazandaran University of Science and Technology,Q2rqevIAAAAJ,master degree in mazandaran university of scie...,ustmb.ac.ir,2,['sna'],History


In [10]:
prof[prof["affiliation"].str.contains("|".join(["Professor"]), case=False)]

,id,name,university,user_id,affiliation,v_email_at,cited_by,intrests,department
0,0,Ali Fahim,University of Tehran,_C4Iif8AAAAJ,"Assistant Professor of Physics, University of ...",ut.ac.ir,121695,"['Elementary Particle Physics', 'Physics of Co...",Physics
1,1,Mohammad Reza Ganjali,University of Tehran,IJ3XCecAAAAJ,"Professor of Analytical Chemistry, University ...",khayam.ut.ac.ir,49589,"['Sensor and biosensor', 'Elctroanalytical Che...",Chemistry
2,2,Norouzi Parviz,University of Tehran,usWI6hMAAAAJ,"Professor of Electrochemistry, Collage of Scie...",ut.ac.ir,24867,"['Electrochemistry', 'Electrochemical Sensors'...",Biology
3,3,Reza Tavakkoli-Moghaddam,University of Tehran,gqVUx4cAAAAJ,Distinguished Professor of Industrial Engineer...,ut.ac.ir,23651,"['Facilities design', 'Supply chain', 'Schedul...",History
5,5,Ali Akbar Saboury,University of Tehran,GZOeVgUAAAAJ,"Professor of Biophysical Chemistry, University...",ut.ac.ir,16497,"['Biothermodynamics', 'Enzyme Kinetics', 'Calo...",Biology
...,...,...,...,...,...,...,...,...,...
49370,49370,Mohsen Pourfallah,Mazandaran University of Science and Technology,p8y8xvkAAAAJ,"Associate Professor of Mechanical Engineering,...",ustmb.ac.ir,600,"['Renewable Energies', 'Energy Management', 'H...",Economics
49371,49371,Shahruz Nasirian,Mazandaran University of Science and Technology,2FCyGy0AAAAJ,Associate Professor at Mazandaran University o...,ustmb.ac.ir,421,"['Nano Physics', 'Nano electronics', 'Gas sens...",Physics
49373,49373,Ali ‌Tajdin,Mazandaran University of Science and Technology,aTLGjt8AAAAJ,"Associate Professor of Industrial Engineering,...",ustmb.ac.ir,319,"['Operation Research', 'Fuzzy Logic', 'Statist...",Mathematics
49377,49377,Soolmaz Abbasalizadeh,Mazandaran University of Science and Technology,XaHe3MgAAAAJ,"Assistant Professor of Electronic Engineering,...",ustmb.ac.ir,84,"['Analog Integrated Circuits', 'Low Power-Low ...",History


In [9]:
prof[prof["affiliation"].str.contains("|".join(["Faculty", "Assistant Professor", "Assosiate Professor", "Full Professor"]), case=False)]

,id,name,university,user_id,affiliation,v_email_at,cited_by,intrests,department
0,0,Ali Fahim,University of Tehran,_C4Iif8AAAAJ,"Assistant Professor of Physics, University of ...",ut.ac.ir,121695,"['Elementary Particle Physics', 'Physics of Co...",Physics
10,10,S. Ali Torabi,University of Tehran,4hlKJCkAAAAJ,"Full Professor, University of Tehran",ut.ac.ir,12227,"['Supply Chain Resilience', 'Business Continui...",Biology
14,14,Mehdi Mehrpooya,University of Tehran,0gS89gsAAAAJ,"Full Professor, University of Tehran.",ut.ac.ir,11515,"['Hydrogen production/storage', 'Hybrid energy...",Chemistry
39,39,Seyed Hassan Jafari,University of Tehran,quIi4pcAAAAJ,Full Professor of Polymer Science and Engineer...,ut.ac.ir,7130,"['Polymer Materials', 'Structure-property rela...",History
70,70,Roham Rafiee,University of Tehran,Q_2ahxsAAAAJ,"Full Professor of Mech. Eng., University of Te...",ut.ac.ir,4566,"['Nano-composites', 'Composite Structures', 'M...",Mathematics
...,...,...,...,...,...,...,...,...,...
49347,49347,mohammad ali Bagherzadeh,Graduate University of Advanced Technology,7B8pHtsAAAAJ,"Assistant Professor, ECE Department, Graduate...",kgut.ac.ir,63,"['Control systems', 'Switched systems', 'Robus...",Computer Science
49351,49351,Hojjatullah Moradi shahrebabaki,Graduate University of Advanced Technology,jofHnAUAAAAJ,Faculty of Electrical and Computer Engineering...,kgut.ac.ir,39,"['nonlinear control and systems', 'systems bio...",Biology
49359,49359,Seyed Hadi Dehghan-Manshadi,Graduate University of Advanced Technology,FGTAZfYAAAAJ,"Assistant Professor of Geophysics, Department ...",kgut.ac.ir,10,"['Engineering Seismology', 'Seismic Hazard Ass...",Mechanical Engineering
49377,49377,Soolmaz Abbasalizadeh,Mazandaran University of Science and Technology,XaHe3MgAAAAJ,"Assistant Professor of Electronic Engineering,...",ustmb.ac.ir,84,"['Analog Integrated Circuits', 'Low Power-Low ...",History


#### Step 0: Calculate h-index and i10-index for professors

In [3]:
prof_citations = {row['id']: [] for idx, row in prof.iterrows()}

# Populate the citation counts for each professor
for idx, art in article.iterrows():
    cite_count = art['cite']
    authors = str(art['link_ids']).split(',')
    for author_id in authors:
        try:
            id = int(author_id)
            if id in prof_citations:
                prof_citations[id].append(cite_count)
        except:
            pass


In [4]:
def calculate_h_index(citations):
    citations.sort(reverse=True)
    h_index = 0
    for i, cite in enumerate(citations):
        if cite >= i + 1:
            h_index = i + 1
        else:
            break
    return h_index

def calculate_i10_index(citations):
    return sum(1 for cite in citations if cite >= 10)

# Calculate h-index and i10-index for each professor
results = []
for prof_id, citations in prof_citations.items():
    h_index = calculate_h_index(citations)
    i10_index = calculate_i10_index(citations)
    results.append({
        'professor_id': prof_id,
        'h_index': h_index,
        'i10_index': i10_index
    })

results_df = pd.DataFrame(results)

In [5]:
prof = prof.merge(results_df, left_on='id', right_on='professor_id', how='left')
prof.drop(columns=["professor_id"], inplace=True)

In [6]:
uni.is_governmental.fillna(0, inplace=True)
uni.is_governmental = uni.is_governmental.astype('b')

#### Step 1: filter universities [keep govermental and non-medical universities]

In [7]:
uni = uni[uni.is_governmental == True]
uni_list = uni.University.to_list()
uni_list[:5]

['University of Tehran',
 'Sharif University of Technology',
 'Ferdowsi University of Mashhad',
 'Amirkabir University of Technology',
 'Shahid Beheshti University']

#### Step 2: filter professors [keep those who are in filtered universities - keep profs with citation more than 100]

In [8]:
del_prof = prof[~ prof["university"].isin(uni_list)]
prof = prof[prof["university"].isin(uni_list)]


prof = prof[prof["cited_by"] > 100]
prof

,id,name,university,user_id,affiliation,v_email_at,cited_by,intrests,department,h_index,i10_index
0,0,Ali Fahim,University of Tehran,_C4Iif8AAAAJ,"Assistant Professor of Physics, University of ...",ut.ac.ir,121695,"['Elementary Particle Physics', 'Physics of Co...",Physics,165,348
1,1,Mohammad Reza Ganjali,University of Tehran,IJ3XCecAAAAJ,"Professor of Analytical Chemistry, University ...",khayam.ut.ac.ir,49589,"['Sensor and biosensor', 'Elctroanalytical Che...",Chemistry,106,994
2,2,Norouzi Parviz,University of Tehran,usWI6hMAAAAJ,"Professor of Electrochemistry, Collage of Scie...",ut.ac.ir,24867,"['Electrochemistry', 'Electrochemical Sensors'...",Biology,80,520
3,3,Reza Tavakkoli-Moghaddam,University of Tehran,gqVUx4cAAAAJ,Distinguished Professor of Industrial Engineer...,ut.ac.ir,23651,"['Facilities design', 'Supply chain', 'Schedul...",History,76,479
4,4,Ali Akbar Moosavi-Movahedi,University of Tehran,sZysKwIAAAAJ,University of Tehran,ut.ac.ir,16918,['Biophysical Chemistry'],Chemistry,63,416
...,...,...,...,...,...,...,...,...,...,...,...
49372,49372,azizollah ardeshir-behrestaghi,Mazandaran University of Science and Technology,J0HyouoAAAAJ,Mazandaran university of science and technology,ustmb.ac.ir,400,[],unknown,9,9
49373,49373,Ali ‌Tajdin,Mazandaran University of Science and Technology,aTLGjt8AAAAJ,"Associate Professor of Industrial Engineering,...",ustmb.ac.ir,319,"['Operation Research', 'Fuzzy Logic', 'Statist...",Mathematics,8,6
49374,49374,Mahbod Armin,Mazandaran University of Science and Technology,2GTMJNoAAAAJ,Researcher at Mazandaran University of Science...,ustmb.ac.ir,244,"['Combustion Simulation & Engine\xa0…', 'Heat ...",Physics,6,5
49375,49375,Aydin Aghajani,Mazandaran University of Science and Technology,QhAn1zUAAAAJ,Mazandaran university of Science and technology,ustmb.ac.ir,201,"['Manufacturing systems', 'Reliability', 'Larg...",Mechanical Engineering,5,4


#### Step 3: filter article [keep (2020 to 2022 articles - filterd professors articles)]

In [9]:
article = article[(2020 <= article.year) & (article.year <= 2022)]

In [10]:
del_prof_ids = set(del_prof.id.to_list())

article = article.dropna(subset='link_ids')

for index, row in article.iterrows():
    link_ids = set(row["link_ids"].split(','))
    new_link_ids = link_ids - del_prof_ids
    article.at[index, "link_ids"] = ','.join(new_link_ids)

In [11]:
article

,title,GS_link,year,cite,main_authors,more_info,link_ids
74,Timing of surgery following SARS‐CoV‐2 infecti...,/citations?view_op=view_citation&hl=en&user=_C...,2021,382,"COVIDSurg Collaborative, GlobalSurg Collaborat...","Anaesthesia 76 (6), 748-758, 2021","7901,0, 7913, 16808, 7749, 7730, 116, 7710, 7..."
288,SARS-CoV-2 infection and venous thromboembolis...,/citations?view_op=view_citation&hl=en&user=_C...,2022,48,"C COVIDSurg, C GlobalSurg","Anaesthesia 77 (1), 28-39, 2022","0, 16808, 7749, 7779, 7730, 7710, 116, 7691, 1..."
323,Effects of pre‐operative isolation on postoper...,/citations?view_op=view_citation&hl=en&user=_C...,2021,24,"COVIDSurg Collaborative, GlobalSurg Collaborat...","Anaesthesia 76 (11), 1454-1464, 2021","7901,0, 7913, 16808, 7749, 7730, 7779, 116, 7..."
387,Measurements of the ...,/citations?view_op=view_citation&hl=en&user=_C...,2021,2,"V Khachatryan, AM Sirunyan, A Tumasyan, W Adam...","Springer Nature, 2021","5600,0, 34234"
399,Parameter identification of complex network dy...,/citations?view_op=view_citation&hl=en&user=_C...,2021,1,"A Bakhtiarnia, A Fahim, EM Miandoab","Nonlinear Dynamics 104 (4), 3991-4005, 2021","763,0"
...,...,...,...,...,...,...,...
2099744,Pattern of analysis of social networks with bi...,/citations?view_op=view_citation&hl=en&user=nZ...,2020,0,"IM F Akbari, H Mirzagoltabar",Fifth International Conference on Science and ...,49376
2099767,Fuzzy Scheduling Problem on Unrelated Parallel...,/citations?view_op=view_citation&hl=en&user=iK...,2020,3,"J Rezaeian, S Mohammad-Hosseini, S Zabihzadeh,...","Artificial Intelligence Evolution 1 (1), 17-33...",49378
2099805,Clustering students’ writing behaviors using k...,/citations?view_op=view_citation&hl=en&user=SQ...,2022,6,"M Talebinamvar, F Zarrabi","Language Testing in Asia 12 (1), 6, 2022",49384
2099818,A Survey of A li ations of Arti ial Intelligen...,/citations?view_op=view_citation&hl=en&user=LQ...,2021,0,"JH Joloudari, S Mojrian, I Nodehi, A Mashmool,...",NaN,49386


#### Step 4: Assign subject to each article based on title with bert model

In [12]:
subjects = pd.read_csv("../build/classified_articles.csv", index_col=0)
article = article.merge(subjects, on='title', how='left')

#### Step 5: calculate Deapth and Breadth

In [ ]:
uni.to_csv("../build/universities.csv")
prof.to_csv("../build/professors.csv")
article.to_csv("../build/articles.csv")

In [2]:
import pandas as pd
article = pd.read_csv("../build/articles.csv")

In [3]:
article[article["title"].str.startswith("A novel method of spectral")]["classified_categories"].values

array(["[('Energy', 0.07741662114858627), ('Mathematics', 0.07584677636623383), ('Decision Sciences', 0.059227220714092255)]"],
      dtype=object)

In [4]:
article[article["title"].str.startswith("Spectral clustering on protein-protein")]["classified_categories"].values

array(["[('Mathematics', 0.08555509150028229), ('Energy', 0.06969840824604034), ('Biochemistry, Genetics and Molecular Biology', 0.06652741134166718)]"],
      dtype=object)

In [8]:
article[article["title"].str.startswith("A novel method of spectral")]['title'].iloc[0]

'A novel method of spectral clustering in attributed networks by constructing parameter-free affinity matrix'

In [9]:
article

,Unnamed: 0,title,GS_link,year,cite,main_authors,more_info,link_ids,classified_categories
0,0,Timing of surgery following SARS‐CoV‐2 infecti...,/citations?view_op=view_citation&hl=en&user=_C...,2021,382,"COVIDSurg Collaborative, GlobalSurg Collaborat...","Anaesthesia 76 (6), 748-758, 2021","7901,0, 7913, 16808, 7749, 7730, 116, 7710, 7...","[('Medicine', 0.2003847062587738), ('Health Pr..."
1,1,Timing of surgery following SARS‐CoV‐2 infecti...,/citations?view_op=view_citation&hl=en&user=_C...,2021,382,"COVIDSurg Collaborative, GlobalSurg Collaborat...","Anaesthesia 76 (6), 748-758, 2021","7901,0, 7913, 16808, 7749, 7730, 116, 7710, 7...","[('Medicine', 0.2003847062587738), ('Health Pr..."
2,2,SARS-CoV-2 infection and venous thromboembolis...,/citations?view_op=view_citation&hl=en&user=_C...,2022,48,"C COVIDSurg, C GlobalSurg","Anaesthesia 77 (1), 28-39, 2022","0, 16808, 7749, 7779, 7730, 7710, 116, 7691, 1...","[('Medicine', 0.23442892730236053), ('Immunolo..."
3,3,Effects of pre‐operative isolation on postoper...,/citations?view_op=view_citation&hl=en&user=_C...,2021,24,"COVIDSurg Collaborative, GlobalSurg Collaborat...","Anaesthesia 76 (11), 1454-1464, 2021","7901,0, 7913, 16808, 7749, 7730, 7779, 116, 7...","[('Medicine', 0.2182307094335556), ('Decision ..."
4,4,Measurements of the ...,/citations?view_op=view_citation&hl=en&user=_C...,2021,2,"V Khachatryan, AM Sirunyan, A Tumasyan, W Adam...","Springer Nature, 2021","5600,0, 34234","[('Mathematics', 0.07036911696195602), ('Busin..."
...,...,...,...,...,...,...,...,...,...
350709,350709,Pattern of analysis of social networks with bi...,/citations?view_op=view_citation&hl=en&user=nZ...,2020,0,"IM F Akbari, H Mirzagoltabar",Fifth International Conference on Science and ...,49376,"[('Health Professions', 0.22171346843242645), ..."
350710,350710,Fuzzy Scheduling Problem on Unrelated Parallel...,/citations?view_op=view_citation&hl=en&user=iK...,2020,3,"J Rezaeian, S Mohammad-Hosseini, S Zabihzadeh,...","Artificial Intelligence Evolution 1 (1), 17-33...",49378,"[('Computer Science', 0.5304924249649048), ('E..."
350711,350711,Clustering students’ writing behaviors using k...,/citations?view_op=view_citation&hl=en&user=SQ...,2022,6,"M Talebinamvar, F Zarrabi","Language Testing in Asia 12 (1), 6, 2022",49384,"[('Energy', 0.08040671795606613), ('Decision S..."
350712,350712,A Survey of A li ations of Arti ial Intelligen...,/citations?view_op=view_citation&hl=en&user=LQ...,2021,0,"JH Joloudari, S Mojrian, I Nodehi, A Mashmool,...",NaN,49386,"[('Health Professions', 0.12294503301382065), ..."
